In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [3]:
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
model = P3alphaRecommender

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

EvaluatorHoldout: Ignoring 2635 (20.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [5]:
import numpy as np 
import scipy.sparse as sps
profile_length = np.ediff1d(sps.csr_matrix(URM_train_validation).indptr)

#profile_length = np.ediff1d(sps.csr_matrix(URM_train_validation).indptr)
profile_length, profile_length.shape

(array([35, 27,  4, ...,  6, 22, 63], dtype=int32), (12638,))

In [6]:
# PROFILE LENGTH BASED GROUPING
from Evaluation.Evaluator import EvaluatorHoldout
import numpy as np

MAP_recommender_per_group = {}
cutoff = 10
threshold = 13

# Select users whose profile length is less than the cutoff for the group and greater than or equal to the cutoff for the previous group

users_in_group_flag = profile_length > threshold
users_in_group = np.arange(len(profile_length))[users_in_group_flag]

users_in_group_p_len = profile_length[users_in_group]

# Select users not in the current group
users_not_in_group_flag = np.isin(np.arange(len(profile_length)), users_in_group, invert=True)
users_not_in_group = np.arange(len(profile_length))[users_not_in_group_flag]

evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_not_in_group)

EvaluatorHoldout: Ignoring 2207 (17.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 6095 Users


In [9]:
import optuna as op
def objective(trial):
    # Optuna suggest params
    topK = trial.suggest_int("topK", 10, 1000)
    alpha = trial.suggest_uniform("alpha", 0.001, 2.0)
    normalize_similarity = trial.suggest_categorical("normalize_similarity", [True, False])
    
    recommender = model(URM_train_validation)
    recommender.fit(**trial.params)
    
    result, _ = evaluator_test.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [11]:
best_params = {
    "topK": 76,
    "alpha": 0.377201600381895,
    "normalize_similarity": True,
}
#{"topK": 84, "alpha": 0.3904145688038595, "normalize_similarity": true}
study_name = "p3alpha2-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=100)

[I 2024-01-10 22:39:18,088] Using an existing study with name 'p3alpha2-study' instead of creating a new one.
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_11921/144902340.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0.001, 2.0)


P3alphaRecommender: URM Detected 207 ( 1.6%) users with no interactions.
P3alphaRecommender: URM Detected 119 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 7483.89 column/sec. Elapsed time 2.97 sec
EvaluatorHoldout: Processed 6514 (100.0%) in 1.60 sec. Users per second: 4060


[I 2024-01-10 22:39:23,019] Trial 100 finished with value: 0.06682690152102219 and parameters: {'topK': 31, 'alpha': 0.3598623800610297, 'normalize_similarity': True}. Best is trial 100 with value: 0.06682690152102219.


P3alphaRecommender: URM Detected 207 ( 1.6%) users with no interactions.
P3alphaRecommender: URM Detected 119 ( 0.5%) items with no interactions.


/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_11921/144902340.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0.001, 2.0)


...

[I 2024-01-10 22:48:26,005] Trial 198 finished with value: 0.06482626552821567 and parameters: {'topK': 83, 'alpha': 0.23648758099574446, 'normalize_similarity': True}. Best is trial 191 with value: 0.06685656290967963.


P3alphaRecommender: URM Detected 207 ( 1.6%) users with no interactions.
P3alphaRecommender: URM Detected 119 ( 0.5%) items with no interactions.


/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_11921/144902340.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = trial.suggest_uniform("alpha", 0.001, 2.0)


P3alphaRecommender: Similarity column 22222 (100.0%), 7047.04 column/sec. Elapsed time 3.15 sec
EvaluatorHoldout: Processed 6514 (100.0%) in 2.15 sec. Users per second: 3027


[I 2024-01-10 22:48:31,602] Trial 199 finished with value: 0.042108724310520367 and parameters: {'topK': 10, 'alpha': 0.1834585348162417, 'normalize_similarity': False}. Best is trial 191 with value: 0.06685656290967963.


In [5]:
study.best_params

{'topK': 76, 'alpha': 0.377201600381895, 'normalize_similarity': True}

In [6]:
final = model(URM_train_validation)
final.fit(**study.best_params)

P3alphaRecommender: URM Detected 225 ( 1.8%) users with no interactions.
P3alphaRecommender: URM Detected 119 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 4880.62 column/sec. Elapsed time 4.55 sec


In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

In [ ]:
## MAP = .046439